In [ ]:
import requests
import pandas as pd
# interactive pandas tables
%load_ext google.colab.data_table
import os
import shutil
try:
    import git
except ImportError:
    print("GitPython is not installed. Installing...")
    !pip install gitpython
    import git
import time
import zipfile

# Download text files with FEC committee id information. These wil be paired with the fec_id from the results dataframe to connect the financial data from the api to the existing dataframe.
### https://www.fec.gov/data/browse-data/?tab=bulk-datadataframes.

In [ ]:
def ccl(year, source_url, download_directory, last_two_digits):

  filename = source_url.split('/')[-1]

  # Send a GET request to the URL
  response = requests.get(source_url)

  if response.status_code == 200:
      # Save the content of the response (the zip file) to a file
      with open(filename, 'wb') as f:
          f.write(response.content)

  else:
      pass

  zip_file_name = f'ccl{last_two_digits}.zip'
  extract_directory = '/raw'

  with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    for file_info in zip_ref.infolist():

      # Extract all files in the zip file
      zip_ref.extractall(extract_directory)

      # Custom name for the extracted file (assuming it's a text file)
      custom_name = f"linkages_{year}.txt"  # Change this to whatever custom name you want

      # Rename the extracted file
      extracted_file_path = os.path.join(extract_directory, file_info.filename)
      new_file_path = os.path.join(extract_directory, custom_name)
      os.rename(extracted_file_path, new_file_path)

      # List the extracted files
      extracted_files = os.listdir('/raw')
      extracted_files

In [ ]:
years = list(range(2004,2028))[::2]
download_directory = "/raw"
for year in years:
  year = str(year)

  last_two_digits = year[-2:]

  source_url = f'https://www.fec.gov/files/bulk-downloads/{year}/ccl{last_two_digits}.zip'
  response = requests.get(source_url)
  if response.status_code == 200:
      ccl(year, source_url, download_directory, last_two_digits)
      print(f'{year} downloaded successfully')
  else:
      print(f"They havn't released info for {year} yet!")

In [ ]:
os.chdir('/raw')

column_names = ['fec_id', 'eh1', 'year', 'committee_id', 'eh2', 'eh3', 'eh4']

# Read the data from each file into a DataFrame
df1 = pd.read_csv('linkages_2016.txt', sep='|', names=column_names)
df2 = pd.read_csv('linkages_2018.txt', sep='|', names=column_names)
df3 = pd.read_csv('linkages_2020.txt', sep='|', names=column_names)

# Concatenate the DataFrames verticallyv
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.drop(['eh1', 'eh2', 'eh3', 'eh4'], axis=1, inplace=True)
df2 = combined_df[combined_df['fec_id'].str.startswith('S')]
# Display the combined DataFrame
df2.head(5)


# keep only the committess that match with fec_id numbers (candidiates) in our results dataframe.
## download the excel file using the "download_senate_results.ipynb" file

In [ ]:
senate_results = pd.read_excel("senate_results.xlsx")
unique_fec_ids = senate_results['fec_id'].unique().tolist()
linkages_df = df2[df2['fec_id'].isin(unique_fec_ids)]
linkages_df.head(5)

# create a list of committee id abnd year pairs to iterate through for data collection.

In [ ]:
# Initialize an empty list to store the pairs
pairs_list = []

# Iterate over each row in the DataFrame
for index, row in linkages_df.iterrows():
    # Extract 'year' and 'committee_id' values for the current row
    year = row['year']
    committee_id = row['committee_id']

    # Append the pair to the list as a tuple
    pairs_list.append((year, committee_id))

# Print or use the resulting list
print(len(pairs_list))

# Use the list to get committee names and financial information by year

# COMMITTEE NAMES

In [ ]:
def get_committee_name(committee_id, year):
    api_key = 'lYvhUilgUTwOu2fyUQF0S7vI8pj1iFD0aLCtkeMH'
    url = f"https://api.open.fec.gov/v1/committee/{committee_id}/totals/?api_key={api_key}"
    response = requests.get(url)
    cycle = committee_name = None
    if response.status_code == 200:
        data = response.json()
        #cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
        for r in data['results']:
          cycle = r['cycle'] #'coverage_end_date': '2022-12-31T00:00:00', 'coverage_start_date': '2021-01-01T00:00:00'
          if cycle == year:

            committee_name = r['committee_name']


            break
          else:
            pass
    else:
        pass

    return committee_name


In [ ]:
%%capture

# Create an empty list to store the results
committee_names_list = []
failed = 0
# Iterate over each pair in pairs_list
for pair in pairs_list:
    year, committee_id = pair

    x = 0
    while True:

        committee_name = get_committee_name(committee_id, year)

        if committee_name is not None:
            print('successful')
            # If committee_name is not None, break the loop
            break
        #time.sleep(1.1)
        print("try again")
        if x > 2:
          failed += 1
          break
        x += 1

    # Call the get_committee_name function with committee_id and year
    #committee_name = get_committee_name(committee_id, year)
    # Append the result to committee_names_list

    committee_names_list.append(committee_name)
    print((len(committee_names_list) - failed), '/', len(committee_names_list))
#committee_names_list

In [ ]:
 len(committee_names_list)|

In [ ]:
linkages_df['committee_name'] = committee_names_list
#linkages_df.to_excel("linkages_with_names.xlsx", index=False)

# AMOUNT RAISED

In [ ]:
def get_committee_amount(committee_id, year):
    api_key = 'lYvhUilgUTwOu2fyUQF0S7vI8pj1iFD0aLCtkeMH'
    url = f"https://api.open.fec.gov/v1/committee/{committee_id}/totals/?api_key={api_key}"
    response = requests.get(url)
    cycle = amount_raised = None
    if response.status_code == 200:
        data = response.json()
        #cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
        for r in data['results']:
          cycle = r['cycle'] #'coverage_end_date': '2022-12-31T00:00:00', 'coverage_start_date': '2021-01-01T00:00:00'
          if cycle == year:
            amount_raised = r['receipts']
            break
          else:
            pass
    else:
        pass
    return amount_raised

In [ ]:
%%capture

# Create an empty list to store the results
import time
total_raised_list = []
failed = 0
# Iterate over each pair in pairs_list
for pair in pairs_list:
    year,committee_id = pair
    x = 0
    while True:

    # Call the get_committee_name function with committee_id and year
      total_raised = get_committee_amount(committee_id, year)

      if total_raised is not None:
            print('success')
            print(total_raised)
            # If committee_name is not None, break the loop
            break
      #time.sleep(1.1)
      if x > 7:
        failed += 1
        #print(x, 'failed')
        break
      x += 1
    # Append the result to committee_names_list
    total_raised_list.append(total_raised)
    print((len(total_raised_list) - failed), '/', len(total_raised_list))
#total_raised_list

In [ ]:
linkages_df['total_raised'] = total_raised_list
#linkages_df.to_excel("linkages_with_raised.xlsx", index=False)

# total spent

In [ ]:
def get_committee_spent(committee_id, year):
    api_key = 'lYvhUilgUTwOu2fyUQF0S7vI8pj1iFD0aLCtkeMH'
    url = f"https://api.open.fec.gov/v1/committee/{committee_id}/totals/?api_key={api_key}"
    response = requests.get(url)
    cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
    if response.status_code == 200:
        data = response.json()
        #cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
        for r in data['results']:
          cycle = r['cycle'] #'coverage_end_date': '2022-12-31T00:00:00', 'coverage_start_date': '2021-01-01T00:00:00'
          if cycle == year:

            disbursements = r['disbursements']

            break
          else:
            pass
    else:
        pass

    return disbursements

In [ ]:
%%capture

# Create an empty list to store the results
total_spent_list = []

failed = 0
# Iterate over each pair in pairs_list
for pair in pairs_list:
    year,committee_id = pair
    x = 0
    while True:

    # Call the get_committee_name function with committee_id and year
      total_spent = get_committee_spent(committee_id, year)

      if total_spent is not None:
            print('success')
            print(total_spent)
            # If committee_name is not None, break the loop
            break
      #time.sleep(1.1)
      if x > 7:
        failed += 1
        break
      x += 1
    # Append the result to committee_names_list
    total_spent_list.append(total_spent)
    print((len(total_spent_list) - failed), '/', len(total_spent_list))
total_spent_list

In [ ]:
len(total_spent_list)

In [ ]:
linkages_df['total_spent'] = total_spent_list
linkages_df.to_excel("linkages_df_with_raised_and_spent.xlsx", index=False)

# To speed things up I will remove rows with missing values (committeess that were not registered on the year privided in the excel)
## Manual work included, filling in the mising values that were skipped over by my code. I added a delay which decreased the amount of missing values greatly, but it takes hours to run to get the collection rate above 90%. i settled on about 10 seconds per committee and out of 1300, only about 30 needed to be filled in manually. This goes for each of committee names, raised, and spent. I then deleted any committee numbers that were not assigned any data (roughly 150 committees didnt exist on the selected year) Then i split the data frame into two, one for multicandidate commites and the other for single candidate commiuttees. Only single candididate coimmittes are considered in the finances totaling.
# It may be useful to include a binary variable representing if the candidate is part of a multicandidate committee.

In [ ]:
linkages_df

In [ ]:
# Initialize an empty list to store the pairs
pairs_list2 = []

# Iterate over each row in the DataFrame
for index, row in linkages_df.iterrows():
    # Extract 'year' and 'committee_id' values for the current row
    year = row['year']
    committee_id = row['committee_id']

    # Append the pair to the list as a tuple
    pairs_list2.append((year, committee_id))

# Print or use the resulting list
print(len(pairs_list2))

# CASH START

In [ ]:
def get_committee_cash_start(committee_id, year):
    api_key = 'lYvhUilgUTwOu2fyUQF0S7vI8pj1iFD0aLCtkeMH'
    url = f"https://api.open.fec.gov/v1/committee/{committee_id}/totals/?api_key={api_key}"
    response = requests.get(url)
    cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
    if response.status_code == 200:
        data = response.json()
        #cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
        for r in data['results']:
          cycle = r['cycle'] #'coverage_end_date': '2022-12-31T00:00:00', 'coverage_start_date': '2021-01-01T00:00:00'
          if cycle == year:

            cash_start = r['cash_on_hand_beginning_period']

            break
          else:
            pass
    else:
        pass

    return cash_start

In [ ]:
%%capture

# Create an empty list to store the results
cash_start_list = []

failed = 0
# Iterate over each pair in pairs_list
for pair in pairs_list2:
    year,committee_id = pair
    x = 0
    while True:

    # Call the get_committee_name function with committee_id and year
      cash_start = get_committee_cash_start(committee_id, year)

      if cash_start is not None:
            print('success')
            # If committee_name is not None, break the loop
            break
      time.sleep(.9)
      if x > 9:
        failed += 1
        break
      x += 1
    # Append the result to committee_names_list
    cash_start_list.append(cash_start)
    print((len(cash_start_list) - failed), '/', len(cash_start_list))

cash_start_list

In [ ]:
len(cash_start_list)

In [ ]:
linkages_df['cash_start'] = cash_start_list
#linkages_df.to_excel("cash_start.xlsx", index=False)

# CASH END

In [ ]:
def get_committee_cash_end(committee_id, year):
    api_key = 'lYvhUilgUTwOu2fyUQF0S7vI8pj1iFD0aLCtkeMH'
    url = f"https://api.open.fec.gov/v1/committee/{committee_id}/totals/?api_key={api_key}"
    response = requests.get(url)
    cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
    if response.status_code == 200:
        data = response.json()
        #cycle = committee_name = amount_raised = disbursements = cash_start = cash_end = None
        for r in data['results']:
          cycle = r['cycle'] #'coverage_end_date': '2022-12-31T00:00:00', 'coverage_start_date': '2021-01-01T00:00:00'
          if cycle == year:
            cash_end = r['last_cash_on_hand_end_period']
            break
          else:
            pass
    else:
        pass
    return cash_end

In [ ]:
%%capture

# Create an empty list to store the results
cash_end_list = []

failed = 0
# Iterate over each pair in pairs_list
for pair in pairs_list2:
    year,committee_id = pair4
    x = 0
    while True:

    # Call the get_committee_name function with committee_id and year
      cash_end = get_committee_cash_end(committee_id, year)

      if cash_end is not None:
            print('success')
            # If committee_name is not None, break the loop
            break
      time.sleep(1.2)
      if x > 8:
        failed += 1
        break
      x += 1
    # Append the result to committee_names_list
    cash_end_list.append(cash_end)
    print((len(cash_end_list) - failed), '/', len(cash_end_list))
cash_end_list

In [ ]:
len(cash_end_list)

In [ ]:
linkages_df['cash_end'] = cash_end_list
linkages_df.to_excel("senate_finances.xlsx", index=False)